Sentence count after removing code  
first line length  after removing code  
shortWordCount    
bodyWordCount  
lowerupperratio  
punctuation Count  

In [1]:
import numpy as np
import pandas as pd
import math
import nltk
import re
import string

In [2]:
#Count of sentences in the post body
def getSentenceCount(text):
    text=removeCode(text)
    return len(split_into_sentences(text))

def getFirstLineLength(text):
    text=removeCode(text)
    sentences=split_into_sentences(text)
    #print(len(sentences))
    if(len(sentences)==0):
        #print(text)
        return 0
    firstLine=sentences[0]
    return len(firstLine)
 
#Removes all occurence of <code>    
def removeCode(text):
    #make sure you don't strip HTML beforehand
    textWithoutCode=re.sub(r"<code>(.*?)</code>","",text,flags= re.DOTALL)
    #print(codes)
    return textWithoutCode

#Removes HTML from query data but doesn't remove the content bw opening and closing tag
#Attributes are removed eg:<a href="this is all removed">not removed</a>
#https://stackoverflow.com/questions/3398852/using-python-remove-html-tags-formatting-from-a-string
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)
   

In [3]:
#Credits: https://stackoverflow.com/questions/4576077/python-split-text-on-sentences

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])" 

def split_into_sentences(text):
    #text=body
    text=striphtml(text)
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    #print(text)
    return sentences

In [26]:
openCloseData=pd.read_csv("openCloseData7.csv")

In [27]:
print(openCloseData.columns)

Index(['Unnamed: 0', 'PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags',
       'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe',
       'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount',
       'closed', 'lowerUpperRatio', 'sentenceCount', 'firstLineLength',
       'codeLength', 'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex'],
      dtype='object')


In [28]:
print(openCloseData.shape)
openCloseData.drop(['Unnamed: 0'],inplace=True,axis=1)
print(openCloseData.shape)

(159996, 41)
(159996, 40)


In [14]:
openCloseData.iloc[9,3]

'<pre><code>$bot = file_get_contents("http://answers.yahooapis.com/AnswersService/V1/questionSearch?appid=API_KEY&amp;query=".$q1."");\r\r\r\r\necho $bot-&gt;Question[0]-&gt;ChosenAnswer;\r\r\r\r\n</code></pre>\r\r\r\r\n\r\r\r\r\n<p>For some reason, the script isn\'t returning anything.</p>\r\r\r\r\n'

In [15]:
removeCode(openCloseData.iloc[9,3])

"<pre></pre>\r\r\r\r\n\r\r\r\r\n<p>For some reason, the script isn't returning anything.</p>\r\r\r\r\n"

In [16]:
getSentenceCount(openCloseData.iloc[9,3])

1

In [17]:
getFirstLineLength(openCloseData.iloc[9,3])

53

In [18]:
openCloseData['sentenceCount']=openCloseData['Body'].apply(getSentenceCount)

In [19]:
print(openCloseData.shape)

(159996, 40)


In [20]:
#openCloseData.to_csv("openCloseData1.csv")

In [20]:
openCloseData.columns

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex'],
      dtype='object')

In [22]:
openCloseData['firstLineLength']=openCloseData['Body'].apply(getFirstLineLength)

In [35]:
print( len(split_into_sentences( removeCode(openCloseData.iloc[3,3]) )[0]) )
print("\n<<<<end>>>\n")
print( len(split_into_sentences(openCloseData.iloc[3,3])[0])  )

210

<<<<end>>>

210


In [36]:
#openCloseData.to_csv("openCloseData2.csv")

In [37]:
print(openCloseData.shape)

(159996, 40)


In [38]:
#https://stackoverflow.com/questions/12628958/remove-small-words-using-python
def countShortWords(text):
    text=removeCode(text)
    text=striphtml(text)
    shortwords = re.findall(r'\W*\b\w{1,3}\b',text)
    #print(shortwords)
    return len(shortwords)


In [39]:
openCloseData['shortWordCount']=openCloseData['Body'].apply(countShortWords)

In [42]:
#openCloseData.to_csv("openCloseData3.csv")

In [43]:
print(openCloseData.shape)

(159996, 40)


In [44]:
openCloseData.columns

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex'],
      dtype='object')

In [45]:
#Returns count of total number of words in the query
#https://stackoverflow.com/questions/19410018/how-to-count-the-number-of-words-in-a-sentence-ignoring-numbers-punctuation-an
def countWords(text):
    text=removeCode(text)
    text=striphtml(text)
    return len(text.split())


In [46]:
openCloseData['bodyWordCount']=openCloseData['Body'].apply(countWords)

In [48]:
#openCloseData.to_csv("openCloseData4.csv")

In [29]:
print(openCloseData.shape)

(159996, 40)


In [49]:
#Returns number of lowercase chars in the text
def lowercaseCount(text):
    text=removeCode(text)
    text=striphtml(text)
    lowercount=0
    try:
        for w in text:
            for char in w:
                if(char.islower()):
                    lowercount+=1
        return lowercount
    except:
        #print(type(text))
        print(text)

#Returns number of uppercase chars in the text
def uppercaseCount(text):
    text=removeCode(text)
    text=striphtml(text)
    uppercount=0
    try:    
        for w in text:
            for char in w:
                if(char.isupper()):
                    uppercount+=1
        return uppercount
    except:
        print(text)
    

In [50]:
openCloseData['lowerUpperRatio']=(openCloseData['Body'].apply(lowercaseCount))/(openCloseData['Body'].apply(uppercaseCount))

In [53]:
#openCloseData.to_csv("openCloseData5.csv")

In [55]:
#Returns count of punctuations in the post body
#https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
def countPunctuations(text):
    #remove all code in the post
    text=removeCode(text)
    #print(text)
    text=striphtml(text)  #will also remove all URLs as URL is mentioned in href=" " attribute
    puncs=re.findall(r'[^\w\s]',text)
    #print(puncs)
    return len(puncs)

print(openCloseData.iloc[104587,3])
print(countPunctuations(openCloseData.iloc[104587,3]))

<p>I have the sample console application whose code is given below</p> <pre><code> static void Main(string[] args) { Console.Writeline( Enter any value ); int x = int.Parse(Console.Readline()); if( x%2 == 0) Console.Writeline( Even Value ); else Console.Writeline( ODD Value ); } </code></pre> <p>I want to display the console of this app in simple window form I have displayed the output line but was unable to figure out sending input to the console application the code for window form</p> <pre><code>var info= new ProcessStartInfo( console.exe ) { UseShellExecute = false RedirectStandardError = true RedirectStandardInput = true RedirectStandardOutput = true } string line= ; using (Process process= Process.Start(info)) { using (StreamReader reader = process.StandardOutput) { while((line = reader.Readline()) != null) Console.Writeline(line); } } </code></pre>
0


In [56]:
openCloseData['punctuationCount']=openCloseData['Body'].apply(countPunctuations)

In [58]:
openCloseData.shape

(159996, 40)

In [59]:
#openCloseData.to_csv('openCloseData6.csv')

In [14]:
openCloseData.iloc[27744,:]

PostId                                                                 20462653
UserId                                                                  3081439
Comment                                                                     103
Body                          <p>In Visual Basic 2010. I have two textbox an...
Title                         how to set a code so that mouse is clicked in ...
Tags                                                                   <vb.net>
Reputation                                                                    3
experienceInTime                                                              0
WebsiteUrl                                                                  NaN
Location                                                                    NaN
AboutMe                                                                     NaN
Views                                                                         1
Upvotes                                 

In [16]:
openCloseData.head()

,PostId,UserId,Comment,Body,Title,Tags,Reputation,experienceInTime,WebsiteUrl,Location,...,websiteUrlFilled,locationFilled,aboutMeFilled,profileImageUrlFilled,tagWeightSum,gunningFog,fleschReadingEase,daleChallReadabilityScore,smogIndex,colemanLiauIndex
0,21170255,5921,102,"<p>I tried to Google for this, but was (surpri...",Can anyone point me to (or post) an array of e...,<javascript><html><arrays><tags>,17527,1953,NaN,"Mountain View, CA",...,0,1,1,0,-207.161031,74.89,-88.22,14.35,0.0,10.18
1,38290647,1601251,102,<p>I am at a decision fork where i have to pic...,RESTful API with either Tomcat or Node.js?,<java><node.js><rest><tomcat><typescript>,151,1425,NaN,NaN,...,0,0,0,0,-536.990588,54.66,-31.38,12.43,0.0,11.45
2,6496114,52924,102,<p>Does anyone know of a good tool to test Sch...,Tool for testing Schema.org markup,<testing><markup><schema.org>,1639,900,http://www.agendarecife.com,"Recife, Brazil",...,1,1,1,0,-160.004636,18.80,37.64,8.35,0.0,24.66
3,25982771,1968462,102,"<p>The <a href=""http://www.postgresql.org/docs...",Inserting into simple Postgres views,<sql><postgresql>,2669,620,https://katrinahoffert.github.io/,"Saskatchewan, Canada",...,1,1,1,1,-188.646160,46.32,-66.24,11.74,0.0,23.40
4,10142243,383479,102,<p>Which frameworks would you recommend for wr...,Frameworks for Unittests for Multithreading in...,<java><multithreading><unit-testing><concurrency>,245,648,NaN,NaN,...,0,0,0,0,-274.409048,21.25,45.43,8.34,0.0,9.47


In [10]:
import re
#Removes all occurence of <code>    
def removeCode(text):
    #make sure you don't strip HTML beforehand
    textWithoutCode=re.sub(r"<code>(.*?)</code>","",text,flags= re.DOTALL)
    #print(codes)
    return textWithoutCode


test="<code>hello</code> hell2o <code>hello2</code>"
print(removeCode(test))

 hell2o 


In [33]:
#Returns count of punctuations in the post body
#https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
def countPunctuations(text):
    #remove all code in the post
    #text=removeCode(text)
    text=re.sub('<code>(.*?)</code>','',text,flags= re.DOTALL)
    #print(text)
    text=striphtml(text)  #will also remove all URLs as URL is mentioned in href=" " attribute
    puncs=re.findall(r'[^\w\s]',text)
    print(puncs)
    return len(puncs)


import pandas as pd
#openCloseData=pd.read_csv("openCloseData.csv")


In [34]:
openCloseData.iloc[6,4]

'<p>I have imported both ArrayList and Integer. In my code I use the symbols in multiple places but I only get an error on one line.</p>\r\r\r\r\n\r\r\r\r\n<p>Here are the import statements:</p>\r\r\r\r\n\r\r\r\r\n<pre><code>import java.util.ArrayList;  \r\r\r\r\nimport java.lang.Integer;\r\r\r\r\n</code></pre>\r\r\r\r\n\r\r\r\r\n<p>Here are the pieces of code that compile properly:</p>\r\r\r\r\n\r\r\r\r\n<pre><code>ArrayList&lt;Integer&gt; primes = new ArrayList&lt;Integer&gt;();\r\r\r\r\nprimes.add(2);\r\r\r\r\nprimes.add(3);\r\r\r\r\nprimes.add(5);\r\r\r\r\nprimes.add(7);\r\r\r\r\n\r\r\r\r\nprivate static void addPrime(ArrayList&lt;Integer&gt; primes)\r\r\r\r\n{\r\r\r\r\n    int newNumber;\r\r\r\r\n    int x;\r\r\r\r\n}\r\r\r\r\n</code></pre>\r\r\r\r\n\r\r\r\r\n<p>This piece of code return the symbol errors listed above:</p>\r\r\r\r\n\r\r\r\r\n<pre><code>while (lastValue &lt; half)\r\r\r\r\n{\r\r\r\r\n    addPrime(ArrayList&lt;Integer&gt; primes);\r\r\r\r\n    lastValue = primes.get

In [35]:
countPunctuations(openCloseData.iloc[6,4])

['.', '.', ':', ':', ':', '.']


6

In [7]:
openCloseData['sentencesStartWithYouCount']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
159966    0
159967    0
159968    0
159969    0
159970    0
159971    0
159972    0
159973    0
159974    0
159975    0
159976    0
159977    0
159978    0
159979    0
159980    0
159981    0
159982    0
159983    0
159984    0
159985    0
159986    0
159987    0
159988    0
159989    0
159990    0
159991    0
159992    0
159993    0
159994    0
159995    0
Name: sentencesStartWithYouCount, Length: 159996, dtype: int64

In [8]:
openCloseData['sentencesStartWithICount']

0         3
1         1
2         0
3         2
4         0
5         2
6         1
7         3
8         1
9         0
10        0
11        2
12        0
13        1
14        2
15        1
16        1
17        0
18        1
19        2
20        1
21        2
22        1
23        0
24        1
25        0
26        2
27        1
28        1
29        2
         ..
159966    2
159967    2
159968    2
159969    0
159970    1
159971    0
159972    0
159973    3
159974    1
159975    2
159976    0
159977    3
159978    1
159979    6
159980    1
159981    4
159982    1
159983    1
159984    3
159985    3
159986    2
159987    1
159988    0
159989    0
159990    0
159991    1
159992    1
159993    1
159994    4
159995    2
Name: sentencesStartWithICount, Length: 159996, dtype: int64

In [21]:
def sentencesStartWithYouCount(text):
    count=0
    text=text.lower()
    #text=re.sub('<.*>','',text)
    text=removeCode(text)
    text=striphtml(text)
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="you"):
            #print("token0="+tokens[0])
            count+=1
    return count;

def sentencesStartWithICount(text):
    count=0
    text=text.lower()
    text=removeCode(text)
    text=striphtml(text)
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="i"):
            #print("token0="+tokens[0])
            count+=1
        if(len(tokens)>=2): #But I don't have, So I switched 
            if(tokens[1]=="i"):
                count+=1
        if(len(tokens)>=3):
            if(tokens[2]=="i"): #As such I had amended...
                count+=1
    return count;

In [10]:
openCloseData['sentencesStartWithYouCount']=openCloseData['Body'].apply(sentencesStartWithYouCount)

In [22]:
openCloseData['sentencesStartWithICount']=openCloseData['Body'].apply(sentencesStartWithICount)

In [24]:
openCloseData.columns

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex'],
      dtype='object')

In [25]:
#openCloseData.to_csv('openCloseData7.csv')

In [29]:
openCloseData.columns

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled', 'tagWeightSum', 'gunningFog',
       'fleschReadingEase', 'daleChallReadabilityScore', 'smogIndex',
       'colemanLiauIndex'],
      dtype='object')

In [30]:
openCloseData['userProfileFilled']=openCloseData['websiteUrlFilled']+openCloseData['locationFilled']+openCloseData['aboutMeFilled']+openCloseData['profileImageUrlFilled']

In [31]:
#openCloseData.to_csv('openCloseData8.csv')